# Sentence Emotion Detection Model

This notebook contains code we used to train our model that uses embedding and LSTM sentiment analysis to predict the emotion of a journal entry (text sentence)

## Preperations

Install SpaCy and other libraries


In [2]:
!pip install --upgrade torch==1.7.1 torchtext==0.8.1 torchvision==0.8.2

     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 7.0MB 24.9MB/s 
     |████████████████████████████████| 12.8MB 245kB/s 
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101


In [3]:
import torch, torchtext
from torch import nn, optim, functional as F
import pandas as pd, csv
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pdb

Import dataset (already cleaned) from dropbox link

In [4]:
!wget -O text.csv https://www.dropbox.com/s/iulhdbo1yc8farq/Emotion_final.csv?dl=0

--2021-04-09 05:24:00--  https://www.dropbox.com/s/iulhdbo1yc8farq/Emotion_final.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6020:18::a27d:4012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/iulhdbo1yc8farq/Emotion_final.csv [following]
--2021-04-09 05:24:01--  https://www.dropbox.com/s/raw/iulhdbo1yc8farq/Emotion_final.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7f7717c0a9a671aec7dcbffd46.dl.dropboxusercontent.com/cd/0/inline/BMRM7u9S6PV0_-ViURzdEs-e3bE8QVAImHMO5Yb2COQEjc-PyF5P5vRJVu0LZwSgciDeTJ1CQqvd0NdEZsAbw0mImxsyeavCfsM944JoP7jqy0S1QBeww12a7CTzBC3xt7_1h36pS-n1zd7Fk6sgJ-Rm/file# [following]
--2021-04-09 05:24:01--  https://uc7f7717c0a9a671aec7dcbffd46.dl.dropboxusercontent.com/cd/0/inline/BMRM7u9S6PV0_-ViURzdEs-e3bE8QVAImHMO5Yb2COQEjc-PyF5P5vRJVu0LZwSgciD

In [5]:
text = pd.read_csv('/content/text.csv')

In [6]:
text

,Number,Text,Emotion
0,1,i didnt feel humiliated,sadness
1,2,i can go from feeling so hopeless to so damned...,sadness
2,3,im grabbing a minute to post i feel greedy wrong,anger
3,4,i am ever feeling nostalgic about the fireplac...,love
4,5,i am feeling grouchy,anger
...,...,...,...
21454,21455,Melissa stared at her friend in dism,fear
21455,21456,Successive state elections have seen the gover...,fear
21456,21457,Vincent was irritated but not dismay,fear
21457,21458,Kendall-Hume turned back to face the dismayed ...,fear


Sentiments into an array for later use

In [7]:
text.Emotion.unique()

array(['sadness', 'anger', 'love', 'surprise', 'fear', 'happy'],
      dtype=object)

In [8]:
sentiment = ['sadness', 'anger', 'love', 'surprise', 'fear', 'happy']

## Dataset

Define Dataset for text and split into train/test subsets

In [9]:
class Sentences(torch.utils.data.Dataset):
    def __init__(self, fn):
        lengths = []
        convert = { u: n for n, u in enumerate(fn['Emotion'].unique()) }
        fn['Emotion'] = fn['Emotion'].apply(lambda u: convert[u])              # 12 unique words should be assigned integers starting from 0
        tokenizer = torchtext.data.utils.get_tokenizer('spacy', 'en_core_web_sm')  # tokenizer using spaCy
        for i in range(len(text['Text'])):
          lengths.append(len(tokenizer(text['Text'].iat[i])))                 # store the number of tokens in each tweet to beused in getitem
        string = ' '.join([text['Text'].iat[i] 
                           for i in range(len(text['Text']))])                # combine everything into one single string
        toks = tokenizer(string)                                                   # tokenize the single string

        self.vocab = torchtext.vocab.build_vocab_from_iterator([toks])
        self.sentiment = fn['Emotion'].values
        self.text = fn['Text'].values
        self.length = lengths
        self.toks = torch.LongTensor([self.vocab[tok] for tok in toks])

    def __len__(self):
        return len(self.length)

    def __getitem__(self, i):
        sum = 0
        for x in range(i):
          sum += self.length[x]
        return (self.sentiment[i], self.toks[sum: sum + self.length[i]])          # return the sentiment and related tokns for a specific tweet

In [10]:
ds_full = Sentences(text)
n_train = int(0.8 * len(ds_full))
n_test = len(ds_full) - n_train
rng = torch.Generator().manual_seed(291)
ds_train, ds_test = torch.utils.data.random_split(ds_full, [n_train, n_test], rng)

1lines [00:00, 20.41lines/s]


Check outputs are correct

In [11]:
ds_test[100]

(5, tensor([   2,   25,   10,   17, 4064,    4, 1331]))

In [12]:
' '.join([ds_full.vocab.itos[x] for x in ds_test[100][1]])

'i am feeling so hyper and bouncy'

In [13]:
sentiment[ds_test[100][0]]

'happy'

In [14]:
len(ds_full.toks)

416914

## Model

Model with embedding and LSTM

In [15]:
class SentenceModel(nn.Module):
      def __init__(self, vocab_size, embedding_dim, lstm_dim, n_cats, n_layers = 2, drop_prob = 0.5):
        super().__init__()                                                      #constructor for parent class
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)          #use word embeddings 
        self.lstm = torch.nn.LSTM(embedding_dim, lstm_dim, n_layers,
                                  dropout=drop_prob, batch_first=True)          #LSTM layer
        self.linear = nn.Linear(lstm_dim, n_cats)
        nn.init.xavier_uniform_(self.embedding.weight.data) #??? need to check this with TA ???
        nn.init.xavier_uniform_(self.linear.weight.data) #??? need to check this with TA ???
        
      def forward(self, text):
        emb = self.embedding(text)
        lstm_out, _ = self.lstm(emb)
        out = self.linear(lstm_out)
        return torch.mean(out, dim=1) # certain dimensions required ??? need to check this with TA ???

Test and Train loops

In [16]:
device = torch.device('cpu')

def run_test(model, ds, crit):
    preds = []                                                                  # array to store predictions
    batch_size = 1                                                              # change batch size here
    model.eval()
    total_loss, total_acc = 0, 0
    ldr = torch.utils.data.DataLoader(ds)
    for labs, txts in ldr:                                                
        labs, txts = labs.to(device), txts.to(device)
        with torch.no_grad():
            outs = model(txts)
            loss = crit(outs, labs)
            total_loss += loss.item()
            total_acc += (outs.argmax(1) == labs).sum().item()
            preds.append(outs.argmax(1))                                        # append all the predictions to an array
    return total_loss / len(ds), total_acc / len(ds), preds, batch_size         # added array return value 'preds' and batchsize

def run_train(model, ds, crit, opt, sched):
    model.train()
    total_loss, total_acc = 0, 0
    ldr = torch.utils.data.DataLoader(ds)
    for labs, txts in ldr:          
        opt.zero_grad()
        labs, txts = labs.to(device), txts.to(device)
        outs = model(txts)
        loss = crit(outs, labs)
        loss.backward()
        opt.step()
        total_loss += loss.item()
        total_acc += (outs.argmax(1) == labs).sum().item()
    sched.step()
    return total_loss / len(ds), total_acc / len(ds)

def run_all(model, test_ds, train_ds, crit, opt, sched, n_epochs=10):
    for epoch in tqdm(range(n_epochs), desc='epochs'):
        train_loss, train_acc = run_train(model, train_ds, crit, opt, sched)
        test_loss, test_acc, _, _ = run_test(model, test_ds, crit)
        tqdm.write(f'epoch {epoch}   train loss {train_loss:.6f} acc {train_acc:.4f}   test loss {test_loss:.6f} acc {test_acc:.4f}')   

## Training

In [19]:
#TEST 1

model = SentenceModel(len(ds_full.vocab), 32, 1, len(text.Emotion.unique()))
device = torch.device('cuda:0') #added GPU since CPU too slow (enable that in notebook settings)
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=1.0)
sched = optim.lr_scheduler.StepLR(opt, 10, gamma=0.1)

run_all(model, ds_test, ds_train, crit, opt, sched, 10)

epoch 0   train loss 1.809199 acc 0.2762   test loss 1.712606 acc 0.3269
epoch 1   train loss 1.809170 acc 0.2764   test loss 1.712604 acc 0.3269
epoch 2   train loss 1.809170 acc 0.2764   test loss 1.712607 acc 0.3269
epoch 3   train loss 1.809170 acc 0.2764   test loss 1.712606 acc 0.3269
epoch 4   train loss 1.809170 acc 0.2764   test loss 1.712605 acc 0.3269
epoch 5   train loss 1.809170 acc 0.2764   test loss 1.712606 acc 0.3269
epoch 6   train loss 1.809170 acc 0.2764   test loss 1.712605 acc 0.3269
epoch 7   train loss 1.809170 acc 0.2764   test loss 1.712605 acc 0.3269
epoch 8   train loss 1.809170 acc 0.2764   test loss 1.712605 acc 0.3269
epoch 9   train loss 1.809170 acc 0.2764   test loss 1.712605 acc 0.3269



In [17]:
#TEST 2

model = SentenceModel(len(ds_full.vocab), 32, 1, len(text.Emotion.unique()))
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=1.0)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=0.1) #step 10->1

run_all(model, ds_test, ds_train, crit, opt, sched, 20)

epoch 0   train loss 1.809194 acc 0.2763   test loss 1.712609 acc 0.3269
epoch 1   train loss 1.606745 acc 0.3163   test loss 1.583358 acc 0.3269
epoch 2   train loss 1.590632 acc 0.3243   test loss 1.576448 acc 0.3269
epoch 3   train loss 1.588923 acc 0.3277   test loss 1.575377 acc 0.3269
epoch 4   train loss 1.588439 acc 0.3277   test loss 1.575337 acc 0.3269
epoch 5   train loss 1.588368 acc 0.3277   test loss 1.575334 acc 0.3269
epoch 6   train loss 1.588358 acc 0.3277   test loss 1.575332 acc 0.3269
epoch 7   train loss 1.588357 acc 0.3277   test loss 1.575334 acc 0.3269
epoch 8   train loss 1.588359 acc 0.3277   test loss 1.575334 acc 0.3269
epoch 9   train loss 1.588360 acc 0.3277   test loss 1.575334 acc 0.3269
epoch 10   train loss 1.588359 acc 0.3277   test loss 1.575334 acc 0.3269
epoch 11   train loss 1.588359 acc 0.3277   test loss 1.575334 acc 0.3269
epoch 12   train loss 1.588360 acc 0.3277   test loss 1.575334 acc 0.3269
epoch 13   train loss 1.588360 acc 0.3277   test

In [18]:
#TEST 3

model = SentenceModel(len(ds_full.vocab), 32, 64, len(text.Emotion.unique())) #lstm_dim 1 -> 64
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=1.0)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=0.1)

run_all(model, ds_test, ds_train, crit, opt, sched, 20)

epoch 0   train loss 1.809693 acc 0.2763   test loss 1.712638 acc 0.3269
epoch 1   train loss 1.606746 acc 0.3163   test loss 1.583356 acc 0.3269
epoch 2   train loss 1.590628 acc 0.3243   test loss 1.576445 acc 0.3269
epoch 3   train loss 1.588917 acc 0.3277   test loss 1.575373 acc 0.3269
epoch 4   train loss 1.588433 acc 0.3277   test loss 1.575333 acc 0.3269
epoch 5   train loss 1.588362 acc 0.3277   test loss 1.575330 acc 0.3269
epoch 6   train loss 1.588352 acc 0.3277   test loss 1.575327 acc 0.3269
epoch 7   train loss 1.588349 acc 0.3277   test loss 1.575324 acc 0.3269
epoch 8   train loss 1.588348 acc 0.3277   test loss 1.575324 acc 0.3269
epoch 9   train loss 1.588348 acc 0.3277   test loss 1.575324 acc 0.3269
epoch 10   train loss 1.588348 acc 0.3277   test loss 1.575324 acc 0.3269
epoch 11   train loss 1.588349 acc 0.3277   test loss 1.575324 acc 0.3269
epoch 12   train loss 1.588348 acc 0.3277   test loss 1.575324 acc 0.3269
epoch 13   train loss 1.588348 acc 0.3277   test

In [19]:
#TEST 4 (BEST SO FAR)

model = SentenceModel(len(ds_full.vocab), 32, 64, len(text.Emotion.unique()))
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=1.0)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=1) #gamma 0.1 -> 1

run_all(model, ds_test, ds_train, crit, opt, sched, 20)

epoch 0   train loss 1.809704 acc 0.2762   test loss 1.712623 acc 0.3269
epoch 1   train loss 1.809252 acc 0.2764   test loss 1.712635 acc 0.3269
epoch 2   train loss 1.809233 acc 0.2764   test loss 1.712613 acc 0.3269
epoch 3   train loss 1.809228 acc 0.2765   test loss 1.712626 acc 0.3269
epoch 4   train loss 1.809226 acc 0.2765   test loss 1.712626 acc 0.3269
epoch 5   train loss 1.809223 acc 0.2765   test loss 1.712630 acc 0.3269
epoch 6   train loss 1.809220 acc 0.2765   test loss 1.712604 acc 0.3269
epoch 7   train loss 1.809220 acc 0.2764   test loss 1.712621 acc 0.3269
epoch 8   train loss 1.809219 acc 0.2764   test loss 1.712647 acc 0.3269
epoch 9   train loss 1.809215 acc 0.2765   test loss 1.712696 acc 0.3269
epoch 10   train loss 1.809222 acc 0.2764   test loss 1.712615 acc 0.3269
epoch 11   train loss 1.809224 acc 0.2764   test loss 1.712678 acc 0.3269
epoch 12   train loss 1.809221 acc 0.2765   test loss 1.712667 acc 0.3269
epoch 13   train loss 1.809232 acc 0.2764   test

In [21]:
#TEST 5

model = SentenceModel(len(ds_full.vocab), 32, 64, len(text.Emotion.unique()))
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=1.0)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=0.0001) #gamma 1 -> 0.0001

run_all(model, ds_test, ds_train, crit, opt, sched, 20)

epoch 0   train loss 1.809709 acc 0.2762   test loss 1.712674 acc 0.3269
epoch 1   train loss 1.685513 acc 0.3277   test loss 1.633857 acc 0.3269
epoch 2   train loss 1.652987 acc 0.3277   test loss 1.633856 acc 0.3269
epoch 3   train loss 1.652986 acc 0.3277   test loss 1.633856 acc 0.3269
epoch 4   train loss 1.652986 acc 0.3277   test loss 1.633856 acc 0.3269
epoch 5   train loss 1.652985 acc 0.3277   test loss 1.633856 acc 0.3269
epoch 6   train loss 1.652986 acc 0.3277   test loss 1.633856 acc 0.3269
epoch 7   train loss 1.652988 acc 0.3277   test loss 1.633856 acc 0.3269
epoch 8   train loss 1.652986 acc 0.3277   test loss 1.633856 acc 0.3269
epoch 9   train loss 1.652986 acc 0.3277   test loss 1.633856 acc 0.3269
epoch 10   train loss 1.652987 acc 0.3277   test loss 1.633856 acc 0.3269
epoch 11   train loss 1.652987 acc 0.3277   test loss 1.633856 acc 0.3269
epoch 12   train loss 1.652986 acc 0.3277   test loss 1.633856 acc 0.3269
epoch 13   train loss 1.652985 acc 0.3277   test

In [20]:
#TEST 6

model = SentenceModel(len(ds_full.vocab), 32, 64, len(text.Emotion.unique()))
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=3.0) #lr 1.0 -> 3.0
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=1)

run_all(model, ds_test, ds_train, crit, opt, sched, 20)

epoch 0   train loss 2.561450 acc 0.2328   test loss 2.465432 acc 0.3269
epoch 1   train loss 2.550762 acc 0.2328   test loss 2.547255 acc 0.3269
epoch 2   train loss 2.550788 acc 0.2328   test loss 2.470872 acc 0.3269
epoch 3   train loss 2.551229 acc 0.2328   test loss 2.464573 acc 0.3269
epoch 4   train loss 2.550795 acc 0.2328   test loss 2.462552 acc 0.3269
epoch 5   train loss 2.550712 acc 0.2328   test loss 2.472194 acc 0.3269
epoch 6   train loss 2.550416 acc 0.2327   test loss 2.463349 acc 0.3269
epoch 7   train loss 2.552428 acc 0.2329   test loss 2.464066 acc 0.3269
epoch 8   train loss 2.550916 acc 0.2328   test loss 2.472926 acc 0.3269
epoch 9   train loss 2.550116 acc 0.2328   test loss 2.499187 acc 0.3269
epoch 10   train loss 2.549954 acc 0.2328   test loss 2.501174 acc 0.3269
epoch 11   train loss 2.550029 acc 0.2328   test loss 2.502170 acc 0.3269
epoch 12   train loss 2.549956 acc 0.2329   test loss 2.501480 acc 0.3269
epoch 13   train loss 2.550230 acc 0.2329   test

In [22]:
#TEST 7

model = SentenceModel(len(ds_full.vocab), 32, 128, len(text.Emotion.unique())) #lstm_dim 64 -> 128
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=1.0)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=1)

run_all(model, ds_test, ds_train, crit, opt, sched, 20)

epoch 0   train loss 1.809891 acc 0.2763   test loss 1.712609 acc 0.3269
epoch 1   train loss 1.809264 acc 0.2765   test loss 1.712634 acc 0.3269
epoch 2   train loss 1.809246 acc 0.2765   test loss 1.712635 acc 0.3269
epoch 3   train loss 1.809242 acc 0.2764   test loss 1.712628 acc 0.3269
epoch 4   train loss 1.809242 acc 0.2765   test loss 1.712622 acc 0.3269
epoch 5   train loss 1.809221 acc 0.2765   test loss 1.712653 acc 0.3269
epoch 6   train loss 1.809227 acc 0.2764   test loss 1.712622 acc 0.3269
epoch 7   train loss 1.809225 acc 0.2764   test loss 1.712643 acc 0.3269
epoch 8   train loss 1.809215 acc 0.2765   test loss 1.712636 acc 0.3269
epoch 9   train loss 1.809212 acc 0.2765   test loss 1.712626 acc 0.3269
epoch 10   train loss 1.809205 acc 0.2764   test loss 1.712622 acc 0.3269
epoch 11   train loss 1.809205 acc 0.2765   test loss 1.712624 acc 0.3269
epoch 12   train loss 1.809210 acc 0.2765   test loss 1.712615 acc 0.3269
epoch 13   train loss 1.809207 acc 0.2764   test

In [23]:
#TEST 8

model = SentenceModel(len(ds_full.vocab), 16, 64, len(text.Emotion.unique())) #embedding_dim 32 -> 16
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=1.0)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=1)

run_all(model, ds_test, ds_train, crit, opt, sched, 20)

epoch 0   train loss 1.809728 acc 0.2763   test loss 1.712628 acc 0.3269
epoch 1   train loss 1.809250 acc 0.2765   test loss 1.712639 acc 0.3269
epoch 2   train loss 1.809241 acc 0.2765   test loss 1.712644 acc 0.3269
epoch 3   train loss 1.809226 acc 0.2765   test loss 1.712641 acc 0.3269
epoch 4   train loss 1.809217 acc 0.2765   test loss 1.712638 acc 0.3269
epoch 5   train loss 1.809213 acc 0.2765   test loss 1.712620 acc 0.3269
epoch 6   train loss 1.809217 acc 0.2765   test loss 1.712643 acc 0.3269
epoch 7   train loss 1.809215 acc 0.2765   test loss 1.712664 acc 0.3269
epoch 8   train loss 1.809216 acc 0.2764   test loss 1.712653 acc 0.3269
epoch 9   train loss 1.809217 acc 0.2765   test loss 1.712630 acc 0.3269
epoch 10   train loss 1.809216 acc 0.2764   test loss 1.712659 acc 0.3269
epoch 11   train loss 1.809215 acc 0.2764   test loss 1.712645 acc 0.3269
epoch 12   train loss 1.809215 acc 0.2765   test loss 1.712651 acc 0.3269
epoch 13   train loss 1.809215 acc 0.2764   test

In [25]:
#TEST 8

model = SentenceModel(len(ds_full.vocab), 64, 64, len(text.Emotion.unique())) #embedding_dim 16 -> 64
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=1.0)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=1)

run_all(model, ds_test, ds_train, crit, opt, sched, 20)

epoch 0   train loss 1.809692 acc 0.2762   test loss 1.712614 acc 0.3269
epoch 1   train loss 1.809240 acc 0.2765   test loss 1.712644 acc 0.3269
epoch 2   train loss 1.809237 acc 0.2764   test loss 1.712643 acc 0.3269
epoch 3   train loss 1.809235 acc 0.2765   test loss 1.712662 acc 0.3269
epoch 4   train loss 1.809222 acc 0.2764   test loss 1.712640 acc 0.3269
epoch 5   train loss 1.809218 acc 0.2765   test loss 1.712658 acc 0.3269
epoch 6   train loss 1.809215 acc 0.2764   test loss 1.712639 acc 0.3269
epoch 7   train loss 1.809214 acc 0.2764   test loss 1.712613 acc 0.3269
epoch 8   train loss 1.809213 acc 0.2765   test loss 1.712615 acc 0.3269
epoch 9   train loss 1.809218 acc 0.2765   test loss 1.712665 acc 0.3269
epoch 10   train loss 1.809218 acc 0.2764   test loss 1.712642 acc 0.3269
epoch 11   train loss 1.809218 acc 0.2765   test loss 1.712635 acc 0.3269
epoch 12   train loss 1.809218 acc 0.2764   test loss 1.712677 acc 0.3269
epoch 13   train loss 1.809223 acc 0.2764   test

TRAINING NOT FINISHED CONTINUE BELOW

**Notes for future training:**


*   Will probably need to add dropout to model to reduce overfitting
*   Try different optimizers and schedulers to hopefully improve accuracy
*   May need to add more epochs since the model may require a large amount of epochs to improve
*   Last resort: remove some classes from dataset so there will be less labels to predict

